In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from load_data import get_pos, get_spikes, get_lfp
from tuning_curves_functions import get_tc_1d, get_odd_firing_idx

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3
import info.R063d4_info as r063d4
import info.R063d5_info as r063d5
import info.R063d6_info as r063d6
import info.R066d1_info as r066d1
import info.R066d2_info as r066d2
import info.R066d3_info as r066d3
import info.R066d4_info as r066d4
import info.R067d1_info as r067d1
import info.R067d2_info as r067d2

In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\'

In [ ]:
info = r063d4

In [ ]:
print(info.session_id)
pos = get_pos(info.pos_mat, info.pxl_to_cm)
csc = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
count_matrix = np.array([[3., 2., 1.],
                         [0., 0., 0.],
                         [1., 0., 2.],
                         [0., 2., 0.]])

In [ ]:
tetrode_mask = np.array([[True, True, False, False],
                         [True, True, False, False],
                         [False, False, True, False],
                         [False, False, False, True]])

In [ ]:
prob = vdm.compute_cooccur(count_matrix, tetrode_mask, num_shuffles=100)
prob['active']

In [ ]:
prob['expected']

In [ ]:
prob['observed']

In [ ]:
assert np.isnan(prob['expected'][0])

In [ ]:
assert np.allclose(prob['expected'][1:], np.array([2. / 3., 0., 1. / 3., 0., (2. / 3. * 1. / 3.)]))

In [ ]:
y = [vdm.SpikeTrain([0.], 'this'),
     vdm.SpikeTrain([0.], 'this'), 
     vdm.SpikeTrain([0.], 'is')]

In [ ]:
howdoesthiswork = np.zeros((len(y), len(y)), dtype=bool)
howdoesthiswork

In [ ]:
def get_tetrode_mask(spikes):
    tetrode_mask = np.zeros((len(spikes), len(spikes)), dtype=bool)
    
    labels = []
    for spiketrain in spikes:
        labels.append(spiketrain.label)
    
    for i, ilabel in enumerate(labels):
        for j, jlabel in enumerate(labels):
            if ilabel == jlabel:
                tetrode_mask[i][j] = True
                
    return tetrode_mask

In [ ]:
tetrode_mask = get_tetrode_mask(y)
print(tetrode_mask)

In [ ]:
y

In [ ]:
labels = []
for spiketrain in y:
    labels.append(spiketrain.label)
print(labels)